# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. Lee  ->  K. Lee  |  ['K. Lee']
J. Liu  ->  J. Liu  |  ['J. Liu']
J. Liu  ->  J. Liu  |  ['J. Liu']
T. Henning  ->  T. Henning  |  ['T. Henning']
Arxiv has 77 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2504.00391


extracting tarball to tmp_2504.00391... done.
Retrieving document from  https://arxiv.org/e-print/2504.00501


extracting tarball to tmp_2504.00501... done.
Retrieving document from  https://arxiv.org/e-print/2504.00536
extracting tarball to tmp_2504.00536... done.
Retrieving document from  https://arxiv.org/e-print/2504.00841


extracting tarball to tmp_2504.00841...

 done.


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure figures/surface_densities.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\textwidth]{figures/surface_densities.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure figures/dust_distribution.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\textwidth]{figures/dust_distribution.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}

Found 123 bibliographic references in tmp_2504.00841/sample631.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.00841-b31b1b.svg)](https://arxiv.org/abs/2504.00841) | **XUE. Thermochemical Modeling Suggests a Compact and Gas-Depleted Structure for a Distant, Irradiated Protoplanetary Disk**  |
|| B. Portilla-Revelo, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2025-04-02*|
|*Comments*| *20 pages, 7 figures, 3 tables. Accepted for publication in The Astrophysical Journal, March 28, 2025*|
|**Abstract**|            Unveiling the physical structure of protoplanetary disk is crucial for interpreting the diversity of the exoplanet population. Until recently, the census of the physical properties of protoplanetary disks probed by mid-infrared observations was limited to the solar neighborhood ($d \lesssim 250$ pc); however, nearby star-forming regions (SFRs) such as Taurus -- where no O-type stars reside -- are not representative of the environments where the majority of the planet formation occurs in the Galaxy. The James Webb Space Telescope (JWST) now enables observations of disks in distant high-mass SFRs, where strong external Far-Ultraviolet (FUV) radiation is expected to impact those disks. Nevertheless, a detailed characterization of externally irradiated disks is still lacking. We use the thermochemical code ProDiMo to model JWST/MIRI spectroscopy and archival visual/near-infrared photometry aiming to constrain the physical structure of the irradiated disk around the solar-mass star XUE 1 in NGC 6357 ($d \approx 1690$ pc). Our findings are: (1) Mid-infrared dust emission features are explained by amorphous and crystalline silicates with compositions similar to nearby disks. (2) The molecular features detected with MIRI originate within the first $\sim 1$ au, consistent with slab models' results. (3) Our model favors a disk truncated at $10$ au with a gas-to-dust ratio of unity in the outskirts. (4) Comparing models of the same disk structure under different irradiation levels, we find that strong external irradiation raises gas temperature tenfold and boosts water abundance beyond $10$ au by a factor of $100$. Our findings suggest the inner disk resists external irradiation, retaining the elements necessary for planet formation.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.00391-b31b1b.svg)](https://arxiv.org/abs/2504.00391) | **Polarization Position Angle Swing and the Rotating Vector Model of Repeating Fast Radio Bursts**  |
|| X. Liu, et al. -- incl., <mark>K. Lee</mark> |
|*Appeared on*| *2025-04-02*|
|*Comments*| *12 pages, 4 figures, submitted*|
|**Abstract**|            Fast radio bursts (FRBs), typically highly polarized, usually have a nearly constant polarization position angle (PA) during each burst. Some bursts show significant PA variations, and one of them was claimed to have a PA variation pattern consistent with the prediction of the rotating vector model (RVM) commonly adopted to fit the PA variations in radio pulsars. We systematically study the PA evolution pattern of 1727 bursts from three active repeating FRB sources monitored by the Five-hundred-meter Aperture Spherical Telescope (FAST). We identify 46 bursts whose PA variations are fully consistent with the RVM. However, the inferred geometrical parameters and rotation periods derived from these RVM fitting are inconsistent from each other. This suggests that the magnetosphere of the FRB central engine is constantly distorted by the FRB emitter, and the magnetic configuration is dynamically evolving.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.00501-b31b1b.svg)](https://arxiv.org/abs/2504.00501) | **Flux variations and torque reversals of Cen X-3**  |
|| <mark>J. Liu</mark> |
|*Appeared on*| *2025-04-02*|
|*Comments*| *9 pages, 8 figs, to appear on ApJ*|
|**Abstract**|            Cen X-3 is an archetypical X-ray pulsar with strong flux variations and alternating torque reversals, both of which are similar to those of recently discovered pulsating ultra-luminous X-ray sources. We study a low state of Cen X-3 occurred in 2023 lasting for $\sim100$ days with Chandra and Insight-HXMT observations, supplemented with MAXI and Fermi/GBM data. The Chandra spectrum during the eclipse of Cen X-3 in the low state is very similar to that in the high state, especially, the Fe lines. The HXMT spectrum in the low state shows an enhanced Fe line, so do the MAXI data. The spin-up/spin-down trends of Cen X-3 are not affected by the low states. All these results indicate that the intrinsic emission in the low states is high, and the low states are just apparently low and are dominated by reprocessed emission. We found that the spin-up to spin-down reversals of Cen X-3 take longer time than the spin-down to spin-up reversals, which provides a definite observation test of any possible torque-reversal models. We discuss insights of these results for understanding the pulsating ultra-luminous X-ray sources.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.00536-b31b1b.svg)](https://arxiv.org/abs/2504.00536) | **The dark side of the universe may be more harmonic than we thought**  |
|| S. Yan, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-04-02*|
|*Comments*| *10 pages, 3 figures*|
|**Abstract**|            The standard paradigm of cosmology assumes two distinct dark components, namely the dark energy driving the late-universe acceleration and the dark matter that is responsible for the structure formation. However, the necessity of splitting the dark-side world into two sectors has not been experimentally or theoretically proven. It is shown in Wang et al. 2024 that cosmology with one unified dark fluid can also explain the cosmic microwave background (CMB) and late-universe data, with the fitting quality not much worse than the standard Lambda cold dark matter ($\Lambda$CDM) model. The present work aims to provide a clearer physical interpretation of the Wang et al. 2024 results. We show that the unified dark fluid model can produce primary CMB temperature and polarization power spectra that are very close to the $\Lambda$CDM prediction (relative difference $\lesssim 10^{-4}$). The model can also mimic the $\Lambda$CDM background expansion history and linear growth factor on sub-horizon scales with percent-level accuracy. With better physical understanding of the model, we make precision tests and find a minor error in the Boltzmann code used in Wang et al. 2024. We correct the error and update the model comparison between $\Lambda$CDM and the unified dark fluid model.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2504.00841/./Tgas_warm.png', 'tmp_2504.00841/./chi_warm.png', 'tmp_2504.00841/./nH2O_warm.png', 'tmp_2504.00841/./Tgas_cold.png', 'tmp_2504.00841/./chi_cold.png', 'tmp_2504.00841/./nH2O_cold.png', 'tmp_2504.00841/./G0=1E0.png', 'tmp_2504.00841/./G0=1E3.png', 'tmp_2504.00841/./G0=1E5.png', 'tmp_2504.00841/./G0=1E0_lines.png', 'tmp_2504.00841/./G0=1E3_lines.png', 'tmp_2504.00841/./G0=1E5_lines.png', '', '', '', '', '', '']
copying  tmp_2504.00841/./Tgas_warm.png to _build/html/
copying  tmp_2504.00841/./chi_warm.png to _build/html/
copying  tmp_2504.00841/./nH2O_warm.png to _build/html/
copying  tmp_2504.00841/./Tgas_cold.png to _build/html/
copying  tmp_2504.00841/./chi_cold.png to _build/html/
copying  tmp_2504.00841/./nH2O_cold.png to _build/html/
copying  tmp_2504.00841/./G0=1E0.png to _build/html/
copying  tmp_2504.00841/./G0=1E3.png to _build/html/
copying  tmp_2504.00841/./G0=1E5.png to _build/html/
copying  tmp_2504.00841/./G0=1E0_lines.png to _build/html/
cop

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\rt}[1]{{#1}}$
$\newcommand{\arraystretch}{1.25}$</div>



<div id="title">

# XUE. Thermochemical Modeling Suggests a Compact and Gas-Depleted Structure for a Distant, Irradiated Protoplanetary Disk

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.00841-b31b1b.svg)](https://arxiv.org/abs/2504.00841)<mark>Appeared on: 2025-04-02</mark> -  _20 pages, 7 figures, 3 tables. Accepted for publication in The Astrophysical Journal, March 28, 2025_

</div>
<div id="authors">

B. Portilla-Revelo, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Unveiling the physical structure of protoplanetary disk is crucial for interpreting the diversity of the exoplanet population. Until recently, the census of the physical properties of protoplanetary disks probed by mid-infrared observations was limited to the solar neighborhood ( $d \lesssim 250$ pc); however, nearby star-forming regions (SFRs) such as Taurus---where no O-type stars reside---are not representative of the environments where the majority of the planet formation occurs in the Galaxy. The James Webb Space Telescope (JWST) now enables observations of disks in distant high-mass SFRs, where strong external Far-Ultraviolet (FUV) radiation is expected to impact those disks.  Nevertheless, a detailed characterization of externally irradiated disks is still lacking. We use the thermochemical code $\prodimo$ to model JWST/MIRI spectroscopy and archival visual/near-infrared photometry aiming to constrain the physical structure of the irradiated disk around the solar-mass star XUE 1 in NGC 6357 ( $d \approx 1690$ pc). Our findings are: (1) Mid-infrared dust emission features are explained by amorphous and crystalline silicates with compositions similar to nearby disks. (2) The molecular features detected with MIRI originate within the first $\sim 1$ au, consistent with slab models' results. (3) Our model favors a disk truncated at $10$ au with a gas-to-dust ratio of unity in the outskirts. (4) Comparing models of the same disk structure under different irradiation levels, we find that strong external irradiation raises gas temperature tenfold and boosts water abundance beyond 10 au by a factor of 100. Our findings suggest the inner disk resists external irradiation, retaining the elements necessary for planet formation.

</div>

<div id="div_fig1">

<img src="tmp_2504.00841/./Tgas_warm.png" alt="Fig6.1" width="16%"/><img src="tmp_2504.00841/./chi_warm.png" alt="Fig6.2" width="16%"/><img src="tmp_2504.00841/./nH2O_warm.png" alt="Fig6.3" width="16%"/><img src="tmp_2504.00841/./Tgas_cold.png" alt="Fig6.4" width="16%"/><img src="tmp_2504.00841/./chi_cold.png" alt="Fig6.5" width="16%"/><img src="tmp_2504.00841/./nH2O_cold.png" alt="Fig6.6" width="16%"/>

**Figure 6. -** Two-dimensional maps in disk's aspect ratio vs radial distance space for selected disk properties. Top row contains the solution from the best representative model for the XUE 1 disk ($\Go=10^5$); bottom row shows the solution from a non-irradiated disk ($\Go=10^0$) with the same gas and dust composition and structure. Left panels: The grey-scale image shows disk gas temperature with isothermal contours in white dashed lines. The color boxes depict the $15\%-85\%$ emitting areas of the lines that are indicated in the upper-left corner of the figure. Middle panels: strength of the UV radiation field in units of the Draine field---dashed contour indicates the region in the disk that is exposed to an FUV strength equal to the value in the solar neighborhood. Solid contour indicates the region where the radial optical depth is one. Right panels: abundance of gas-phase water relative to the total number of hydrogen nuclei---dashed line encloses $90\%$ of the total abundance. The quantitative analyses presented in Sects. \ref{sec:gas-distribution} and \ref{sec:temperatures-and-abundances} are done at the grid point indicated by the cross. (*fig:Tgas*)

</div>
<div id="div_fig2">

<img src="tmp_2504.00841/./G0=1E0.png" alt="Fig7.1" width="16%"/><img src="tmp_2504.00841/./G0=1E3.png" alt="Fig7.2" width="16%"/><img src="tmp_2504.00841/./G0=1E5.png" alt="Fig7.3" width="16%"/><img src="tmp_2504.00841/./G0=1E0_lines.png" alt="Fig7.4" width="16%"/><img src="tmp_2504.00841/./G0=1E3_lines.png" alt="Fig7.5" width="16%"/><img src="tmp_2504.00841/./G0=1E5_lines.png" alt="Fig7.6" width="16%"/>

**Figure 7. -** $\rt${Effect of external irradiation on the physical structure and line emission of the fiducial disk. _Top:_} Two-dimensional distribution of gas-phase \ce{OH} for different realizations of the fiducial disk. Each realization corresponds to a different value of $\Go$. Dashed lines enclose $90\%$ of the total abundance of \ce{OH}. The insets display for each model the ratio of the synthetic spectral index to the observed one and the corresponding mass of \ce{NO} contained beyond $10$ au. $\rt${_Bottom:_ Synthetic spectra for \ce{NO, H2} and \ce{CH3+}. Note the difference on the vertical scale.} (*fig:size-irradiation-degeneracy*)

</div>
<div id="div_fig3">

<img src="" alt="Fig9.1" width="16%"/><img src="" alt="Fig9.2" width="16%"/><img src="" alt="Fig9.3" width="16%"/><img src="" alt="Fig9.4" width="16%"/><img src="" alt="Fig9.5" width="16%"/><img src="" alt="Fig9.6" width="16%"/>

**Figure 9. -** Two-dimensional distribution of gas-phase \ce{OH} for different realizations of the fiducial disk. Each realization corresponds to a different value of $\Go$. Dashed lines enclose $90\%$ of the total abundance of \ce{OH}. The insets display for each model the ratio of the synthetic spectral index to the observed one, and the mass of \ce{NO} contained beyond $10$ au. (*fig:size-irradiation-degeneracy*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.00841"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

419  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

13  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
